# Projet maison n° 3

In [345]:
# imports
import pandas as pd

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python qui produit un unique DataFrame avec tous les fichiers en utilisant pandas (par ex. fonction "concat" ou méthode "append"), pas de bash :)

Ordre et noms des colonnes : 'year', 'name', 'gender', 'births'

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini par les différents fichiers (voir la documentation du dataset).

In [346]:
# names us
import pathlib
import numpy as np

def df_names_us():
    df = pd.DataFrame()
    df_list = []
    
    files_path = pathlib.Path('/Users/elieishak/Desktop/TELECOM-PARIS 2021-2022/I721/Kit-Start-Data-main/Kit-Start-Data-main/session3/names')

#     df = pd.DataFrame(columns=["year","name","gender","births"])
    
    for filename in files_path.iterdir():
        d = pd.read_csv(filename, header=None, names=["name","gender","births"])
        d.insert(0, column = 'year', value = str(filename)[-8:-4])
        df_list.append(d)
        
    df = pd.concat(df_list,axis=0)
    df.fillna('NA' , inplace=True)
    df = df[df.year != 'DS_S']
    df = df.sort_values(by=['year','gender','births','name'], ascending=[True, True, False, True])
    df = df.reset_index(drop=True)
        
    return df

df_names_us()

,year,name,gender,births
0,1880,Mary,F,7065.0
1,1880,Anna,F,2604.0
2,1880,Emma,F,2003.0
3,1880,Elizabeth,F,1939.0
4,1880,Minnie,F,1746.0
...,...,...,...,...
2020858,2020,Zykell,M,5.0
2020859,2020,Zylus,M,5.0
2020860,2020,Zymari,M,5.0
2020861,2020,Zyn,M,5.0


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : year, name, gender, births
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne 'gender'
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [357]:
import pandas as pd

def df_names_fr():
    
    df = pd.read_csv('nat2020.csv', delimiter=";",header=None,skiprows = 1, names = ['gender','name','year','births'],dtype={'gender': 'int', 'name': 'string', 'births': 'int'})
    
    mapping = {'1': 'M', '2': 'F'}
    df["gender"] = df["gender"].map(mapping)
    
    df = df[df.year != 'XXXX']
#     df.year = df.year.astype(int)
    
    df = df[df['name'] != '_PRENOMS_RARES' ]
    df = df[df['name'].apply(len)>=2] 

    df['name']= df.name.str.capitalize()
    df = df[['year', 'name', 'gender', 'births']]
    df = df.sort_values(by=['year', 'name', 'gender', 'births'], ascending=[True, False, False, True])
    df.reset_index(drop=True)
    
    return df

df_names_fr()

,year,name,gender,births
667244,1900,Zulmee,NaN,5
667207,1900,Zulma,NaN,32
303718,1900,Zoé,NaN,6
666171,1900,Zoé,NaN,131
666304,1900,Zoee,NaN,4
...,...,...,...,...
304583,2020,Aaliya,NaN,15
304563,2020,Aalia,NaN,4
304556,2020,Aaleyah,NaN,4
168,2020,Aahil,NaN,5


## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/#/downloading

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3.
 
1. Utiliser le dataset des taux de change fourni.


2. Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction pd.to_datetime() avec le format '%d/%m/%Y'). Les colonnes du DataFrame doivent correspondre aux devises.

### Tests

In [361]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 2020863)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 630407)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test names
        self.assertTrue(df.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df['gender']=='F']) + len(df.loc[df['gender']=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

#     def test_df_taux_change(self):
#         df = df_taux_change(['CHF', 'GBP', 'USD'])
#         # colonnes
#         self.assertEqual(list(df.columns), ['CHF', 'GBP', 'USD'])
#         # index
#         self.assertTrue(isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex))
#         # types taux
#         self.assertTrue((df.dtypes == 'float').all())
#         # test NaN
#         self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

In [362]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [363]:
# run tests

run_tests()

test_df_names_fr (__main__.Lesson4Tests) ... FAIL
test_df_names_us (__main__.Lesson4Tests) ... ok

FAIL: test_df_names_fr (__main__.Lesson4Tests)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-361-072aa624ecef>", line 20, in test_df_names_fr
    self.assertEqual(len(df), 630407)
AssertionError: 630406 != 630407

----------------------------------------------------------------------
Ran 2 tests in 5.993s

FAILED (failures=1)
